In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
df=pd.read_csv('C:/data_science/gridcomputing/Modified_dataset_20001.csv')

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3,random_state = 0)
Y=np.array(df['WaitTime'])
X=df.drop('WaitTime',axis=1)

In [4]:
train.shape,test.shape

((14000, 18), (6001, 18))

In [5]:
train_y=np.array(train['WaitTime'])
train_x=train.drop('WaitTime',axis=1)
test_y=np.array(test['WaitTime'])
test_x=test.drop('WaitTime',axis=1)

In [6]:
EPSILON = 1e-10


def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted


def _percentage_error(actual: np.ndarray, predicted: np.ndarray):
    """
    Percentage error
    Note: result is NOT multiplied by 100
    """
    return _error(actual, predicted) / (actual + EPSILON)


def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    """ Naive forecasting method which just repeats previous samples """
    return actual[:-seasonality]


In [7]:
def mae(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))

In [8]:

def smape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Mean Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))

In [9]:
def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))

def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))

In [10]:
def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))

In [11]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train_x.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               2304      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 167,169
Trainable params: 167,169
Non-trainable params: 0
_________________________________________________________________


In [12]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [13]:
NN_model.fit(train_x, train_y, epochs=15, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Train on 11200 samples, validate on 2800 samples
Epoch 1/15
11200/11200 [==============================] - 2s 138us/step - loss: 334358.5568 - mean_absolute_error: 334358.4062 - val_loss: 44622.2669 - val_mean_absolute_error: 44622.2695

Epoch 00001: val_loss improved from inf to 44622.26692, saving model to Weights-001--44622.26692.hdf5
Epoch 2/15
11200/11200 [==============================] - 1s 83us/step - loss: 28710.1238 - mean_absolute_error: 28710.1309 - val_loss: 10730.2953 - val_mean_absolute_error: 10730.2988

Epoch 00002: val_loss improved from 44622.26692 to 10730.29533, saving model to Weights-002--10730.29533.hdf5
Epoch 3/15
11200/11200 [==============================] - 1s 71us/step - loss: 15416.3003 - mean_absolute_error: 15416.3057 - val_loss: 8834.0116 - val_mean_absolute_error: 8834.0117

Epoch 00003: val_loss improved from 10730.29533 to 8834.01161, saving model to Weights-003--8834.01161.hdf5
Epoch 4/15
11200/11200 [==============================] - 1s 73us/step -

In [14]:
predictions = NN_model.predict(test_x)

In [15]:
mae(test_y,predictions)

6836.400984074056

In [16]:
smape(test_y,predictions)

1.1610362904647986

In [17]:
rmse(test_y,predictions)

43539.272168651114

In [18]:
mase(test_y,predictions)

0.5364978892911283

In [35]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(hidden_layer_sizes = (100, 75, 50, 25), activation = 'tanh', solver = 'sgd', learning_rate = 'adaptive',random_state = 0,max_iter=500,warm_start=True)
reg.fit(train_x, train_y)

#redict(train_x)

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100, 75, 50, 25), learning_rate='adaptive',
             learning_rate_init=0.001, max_iter=500, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=0, shuffle=True, solver='sgd', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=True)

In [36]:
pred = reg.predict(test_x)
mae(test_y,pred)

12563.555339801036

In [37]:
rmse(test_y,pred)

43022.303601787164

In [38]:
smape(test_y,pred)

1.8025440376356203

In [40]:
mase(test_y,pred)

0.9859458123503372